In [1]:
import gc
import os
import time

import polars as pl
import torch

# import warnings
# warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)
if torch.cuda.is_available():
    print('device name:', torch.cuda.get_device_name(device))

EVAL_BATCH_SIZE = 128

def collect(*, verbose=True):
    if verbose:
        print('garbage collector collected %d objects' % gc.collect())
    else:
        gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

collect()

device: cuda
device name: NVIDIA GeForce RTX 3050 Laptop GPU
garbage collector collected 56 objects


In [2]:
news = pl.read_parquet('private-data/filtered_pairs_2022.parquet')

print(len(news), 'rows')

news.head(3)

5946 rows


pair_id,summary_id,article_id,summary,article,simhash_distance,summary_title,article_title
u32,i32,i32,str,str,i64,str,str
1093692,43423102,43422899,"""Tại phiên họp thứ 14 ngày 11/8…","""100% Ủy viên Ủy ban Thường vụ …",16,"""Quốc hội thông qua việc thành …","""Thành lập thị xã Chơn Thành th…"
1718596,43370015,43360018,"""Tập đoàn Điện lực Việt Nam (EV…","""Theo thông tin từ Tập đoàn Điệ…",17,"""Xuất hiện trang web giả mạo th…","""Xuất hiện trang web giả mạo th…"
1258448,44012345,44017658,"""Mưa lớn vừa qua khiến hàng tấn…","""Trận mưa lớn, liên tục trong c…",14,"""Cảnh tan hoang ở nghĩa trang l…","""Xót xa cảnh nghìn ngôi mộ tại …"


In [3]:
from bert_score import score as bert_score

collect()

samples = news.sample(3, seed=2025).select('pair_id', 'summary', 'article')

for i, model_name in enumerate([
    'bert-base-multilingual-cased',
    'facebook/mbart-large-50-many-to-many-mmt',
]):
    print('Calculating BERTScore using %s' % model_name)
    now_ns = time.time_ns()
    precisions, recalls, _ = bert_score(
        samples.select('summary').to_series().to_list(),
        samples.select('article').to_series().to_list(),
        model_type=model_name,
        lang='vi',
        use_fast_tokenizer=True,
        idf=False, 
        batch_size=min(EVAL_BATCH_SIZE, len(samples)),
        verbose=False, 
        device=device,
    )
    delta_s =  round(time.time_ns() - now_ns) / 1_000_000_000
    print('Time taken: %.4f seconds' % delta_s)
    bs = {
        'bs_p': precisions.tolist(),
        'bs_r': recalls.tolist(),
    }
    for score_name, score_values in bs.items():
        samples = samples.with_columns(
            pl
            .Series(score_values)
            .alias(score_name + '_' + str(i))
        )

samples

garbage collector collected 0 objects
Calculating BERTScore using bert-base-multilingual-cased
Time taken: 5.0908 seconds
Calculating BERTScore using facebook/mbart-large-50-many-to-many-mmt
Time taken: 22.5666 seconds


pair_id,summary,article,bs_p_0,bs_r_0,bs_p_1,bs_r_1
u32,str,str,f64,f64,f64,f64
1144795,"""Khi đang lẩn trốn tại một địa …","""Sau hai ngày trốn khỏi trại gi…",0.799396,0.691559,0.990648,0.987232
1796272,"""Cơ quan công an đang điều tra …","""Nguyên nhân vụ việc ban đầu đư…",0.783248,0.624593,0.986422,0.973143
1359180,"""Sáng 5/12, Bộ Chính trị, Ban B…","""Sáng 5-12, Bộ Chính trị, Ban B…",0.800523,0.608533,0.981096,0.985774
